<a href="https://colab.research.google.com/github/farihahahaa/DiscoveryFall22/blob/main/InitDataScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import glob # package that will help us grab all the pdfs in a folder
from pathlib import Path
from tqdm import tqdm
import re
import operator
import math
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
%matplotlib inline

pd.set_option("display.max_columns", 330)
pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings("ignore")

#!pip install pdfplumber -q
# import pdfplumber

In [2]:
!pip install tabula-py
# !pip install tabula-py[jpype]
import tabula

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 16.6 MB/s eta 0:00:00


In [3]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)
test_9_app = "/content/gdrive/MyDrive/Terner/PDFs/17-003.pdf"
#filepath_research1
# research1 = pd.read_csv(filepath_research1, index_col=0)

#test_9_app = "../Downloading_LIHTC_Applications/2017_lihtc_9%/17-001.pdf"

Mounted at /content/gdrive


In [4]:
#documentation here https://tabula-py.readthedocs.io/en/latest/tabula.html
tabula.convert_into(test_9_app, "test_9%.csv", output_format="csv", pages=[22,23])
#pg 22 and 23 is sources and use budget for this app
#tabula generates csvs, so read back in what it generated
tabula_test_output = pd.read_csv("test_9%.csv", header=1)
df = tabula_test_output.copy()
#df.head()

Jun 28, 2024 5:18:54 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Jun 28, 2024 5:18:54 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Jun 28, 2024 5:18:54 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [ ]:
#, 'TOTAL\rPROJECT\rCOST': 'TOTAL PROJECT COST', 'TAX CREDIT\rEQUITY': 'TAX CREDIT EQUITY', '2)Los Angeles\rCounty\rCommunity\rDevelopment\rCommission': '2)Los Angeles County Community Development Commission'

In [19]:
df=df.rename(columns = {'Unnamed: 0':'LAND_COST/ACQUISITION'})
#df.columns = df.columns.str.extract(r'\\r', expand=False)
#df=df.rename(columns = {'Unnamed: 0':'LAND_COST/ACQUISITION'}) .replace(r'/[\r\n]', " ")
# def clean_column_names(columns):
#     return [re.sub(r'\r', '', col) for col in columns]
# df.columns = clean_column_names(df.columns)
df.columns = df.columns.str.replace('\r', '_')

# print("Original Column Names:", [repr(col) for col in df.iloc[0]])
# cleaned_columns = clean_column_names(df.iloc[0])
# print("Cleaned Column Names:", [repr(col) for col in cleaned_columns])
# df.columns = cleaned_columns
df = df.iloc[1:]
df.head(5)

,LAND_COST/ACQUISITION,TOTALPROJECTCOST,RES. COST,COM'L. COST,TAX CREDITEQUITY,1)Citibank,2)Los AngelesCountyCommunityDevelopmentCommission,3)DeferredDeveloper Fee,4)LA CDC FeeWaiver,5)GP Equity,6),7),8),9),10),11),12),SUBTOTAL,70% PVC forNewConst/Rehab,30% PVC forAcquisition
5,1Total Land Cost or Value,"$1,630,000","$1,630,000",NaN,"$1,630,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$1,630,000",NaN,NaN
6,Existing Improvements Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2Off-Site Improvements,"$150,000","$150,000",NaN,"$150,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$150,000","$150,000",NaN
8,Total Acquisition Cost,"$150,000","$150,000",NaN,"$150,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$150,000",NaN,NaN
9,Total Land Cost / Acquisition Cost,"$1,780,000","$1,780,000",NaN,"$1,780,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$1,780,000",NaN,NaN


Following is code to use (uncomment lines to import file)

In [13]:
# !pip install tabula-py
# import tabula
# from google.colab import drive
# drive.mount("/content/gdrive", force_remount=True)
# app = "/content/gdrive/MyDrive/Terner/PDFs/_______.pdf"
#tabula.convert_into(app, "test_9%.csv", output_format="csv", pages=[22,23])
#tabula_test_output = pd.read_csv("test_9%.csv", header=1)
#df = tabula_test_output.copy()

def import_and_clean_table(pdf):
  csv = tabula.convert_into(pdf, f"{pdf}.csv", output_format="csv", pages=[22,23])
  df = pd.read_csv(csv, header=1) #csv.get()
  df=df.rename(columns = {'Unnamed: 0':'LAND_COST/ACQUISITION'})
  def clean_column_names_r(columns):
      return [re.sub(r'\r', '', col) for col in columns]
  df.columns = clean_column_names_r(df.columns)
  df = df.iloc[1:]
  return df

df.head(5)

,LAND_COST/ACQUISITION,TOTALPROJECTCOST,RES. COST,COM'L. COST,TAX CREDITEQUITY,1)Citibank,2)Los AngelesCountyCommunityDevelopmentCommission,3)DeferredDeveloper Fee,4)LA CDC FeeWaiver,5)GP Equity,6),7),8),9),10),11),12),SUBTOTAL,70% PVC forNewConst/Rehab,30% PVC forAcquisition
1,1Land Cost or Value,"$1,600,000","$1,600,000",NaN,"$1,600,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$1,600,000",NaN,NaN
2,2Demolition,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Legal,"$30,000","$30,000",NaN,"$30,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$30,000",NaN,NaN
4,Land Lease Rent Prepayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1Total Land Cost or Value,"$1,630,000","$1,630,000",NaN,"$1,630,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$1,630,000",NaN,NaN


In [23]:
import tabula
import pandas as pd

def extract_table_from_pdf(pdf_path, output_csv_path):
    # Extract tables from the PDF
    #tables = tabula.read_pdf(pdf_path, pages='22-23', multiple_tables=True)
    # csv = tabula.convert_into(pdf_path, f"{pdf_path}.csv", output_format="csv", pages=[22,23])
    # df = pd.read_csv(csv, header=1)

    # # Concatenate tables if more than one table is extracted
    # df = pd.concat(tables, ignore_index=True)

    tabula.convert_into(pdf_path, output_csv_path, output_format="csv", pages='22-23')

    # Read the CSV into a DataFrame
    df = pd.read_csv(output_csv_path)

    # Clean the column names
    df=df.rename(columns = {'Unnamed: 0':'LAND_COST/ACQUISITION'})
    #df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w\s]', '')
    df.columns = df.columns.str.replace('\r', '_')
    df = df.iloc[1:]

    # Save the DataFrame to a CSV file
    df.to_csv(output_csv_path, index=False)

    #csv = tabula.convert_into(pdf, f"{pdf}.csv", output_format="csv", pages=[22,23])
    #df = pd.read_csv(csv.get(), header=1)
    #df=df.rename(columns = {'Unnamed: 0':'LAND_COST/ACQUISITION'})
    # def clean_column_names_r(columns):
    #   return [re.sub(r'\r', '', col) for col in columns]
    # df.columns = clean_column_names_r(df.columns)
    df = df.iloc[1:]

    return df

# Usage
pdf_path = "/content/gdrive/MyDrive/Terner/PDFs/17-004.pdf"
output_csv_path = "/content/gdrive/MyDrive/Terner/CSVs/17-004.csv"
cleaned_df = extract_table_from_pdf(pdf_path, output_csv_path)

# Display the cleaned DataFrame
print(cleaned_df)


    IV. SOURCES AND USES BUDGET - SECTION 1: SOURCES AND USES BUDGET  \
2                                                1Land Cost or Value   
3                                                        2Demolition   
4                                                              Legal   
5                                         Land Lease Rent Prepayment   
6                                          1Total Land Cost or Value   
..                                                               ...   
114                                                              NaN   
115                                                              NaN   
116                                                              NaN   
117                                                              NaN   
118                                                              NaN   

    Permanent Sources  Unnamed: 2 Unnamed: 3  Unnamed: 4 Unnamed: 5  \
2          $4,793,931  $4,793,931        NaN  $4,793,931        

In [15]:
# from google.colab import drive
# drive.mount("/content/gdrive", force_remount=True)
# app4 = "/content/gdrive/MyDrive/Terner/PDFs/17-004.pdf"
# import_and_clean_table(app4)


In [ ]:
# from collections import Counter, defaultdict
# from itertools import combinations
# import pandas as pd
# import numpy as np
# import operator
# import math
# import seaborn as sns
# import itertools
# from sklearn.feature_extraction import DictVectorizer
# from sklearn import preprocessing, tree
# import matplotlib.pyplot as plt
# from sklearn import metrics
# %matplotlib inline
# import zipfile

# import torch

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from tensorflow.keras.optimizers.legacy import Adam
# # Modelling
# from sklearn.model_selection import cross_val_score
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import AdaBoostClassifier
# #https://www.datacamp.com/tutorial/understanding-logistic-regression-python
# from sklearn.ensemble import RandomForestClassifier
# #https://www.datacamp.com/tutorial/random-forests-classifier-python
# from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
# from sklearn.model_selection import RandomizedSearchCV, train_test_split
# from scipy.stats import randint
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# import seaborn as sns